In [1]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
                        LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
                        MarkerCluster, Heatmap,SearchControl, 
                        FullScreenControl)

from ipywidgets import Text, HTML
from branca.colormap import linear
import geopandas as gpd
import pandas as pd
import json
from dotenv import load_dotenv
load_dotenv()
import requests
from requests.auth import HTTPBasicAuth
import os
from shapely.geometry import Point
from geopandas.tools import sjoin

In [ ]:
url = os.getenv('COUCHDB_URL')+"/bike_locations/_design/datetime_doc/_view/datetime-index?limit=10000&include_docs=true&descending=true"

payload={}
response = requests.get(url, data=payload, auth=HTTPBasicAuth(os.getenv('COUCHDB_USER'), os.getenv('COUCHDB_PASSWORD')))

data = [row['doc'] for row in response.json()['rows']]
for d in data:
    d.pop('_id', None)
    d.pop('_rev', None)
bike_locations_df = pd.DataFrame(data)
bike_locations=gpd.GeoDataFrame(bike_locations_df, crs=4326, geometry=gpd.points_from_xy(bike_locations_df.lon, bike_locations_df.lat))
bike_locations.drop(columns=['lat', 'lon'],inplace=True)
print(len(bike_locations.index))
bike_locations.head()

In [ ]:
bike_locations[['operator','id_temp']] = bike_locations['id'].str.split(':', 1, expand=True)
bike_locations.drop(columns=['id_temp'])
bike_locations.head(2)


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
country =  world[world.name == "Norway"]
print(len(country.index))
country.head()

In [ ]:
bikes_in_country = sjoin(bike_locations,country, how='inner')
bikes_in_country.head()

In [ ]:
country.plot()

## Base maps, Markers & Icons


In [ ]:
bikes_in_country.plot()

In [ ]:
bydeler=gpd.read_file('oslo_bydeler/bydeler_xls.shp')
bydeler.head()

In [ ]:
bydeler.plot()

In [ ]:
bikes_in_oslo = sjoin(bike_locations,bydeler, how='left')
bikes_in_oslo.head()

In [ ]:
bydeler_with_bikes = sjoin(bydeler,bike_locations, how='inner')

bydeler_with_bikes.head()

In [ ]:
bike_count_per_bydel=bydeler_with_bikes.groupby('streng')['streng'].value_counts()
bike_count_per_bydel.head()

In [ ]:
dfsjoin = gpd.sjoin(bydeler, bike_locations) #Spatial join Points to polygons
dfpivot = pd.pivot_table(dfsjoin,index='streng',columns='operator',aggfunc={'id':len})
dfpivot.columns = dfpivot.columns.droplevel()

dfpolynew = bydeler.merge(dfpivot, how='left', on='streng').fillna(0)
dfpolynew.head(5)

In [ ]:
max_count=dfpolynew[["YLI","YTI","YVO"]].max().max()
max_count

In [ ]:
dfpolynew.plot(column='YLI',legend=True,vmax=max_count);

In [ ]:
dfpolynew.plot(column='YLI',legend=True,vmax=max_count);

In [ ]:
dfpolynew.plot(column='YVO',legend=True,vmax=max_count);